<a href="https://colab.research.google.com/github/lilyaYAHIAOUI/Sign-Language-Classification/blob/main/Sign_Language_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import urllib.request
import os
import zipfile
import random
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
from shutil import copyfile
import pandas as pd

# Loading data

In [4]:
IMAGE_DIR = "/content/drive/MyDrive/competition/Zindi/Sign Language classification/data/Images"
IMAGE_TEST_DIR = "/content/drive/MyDrive/competition/Zindi/Sign Language classification/data/Test"
IMAGE_TRAIN_DIR = "/content/drive/MyDrive/competition/Zindi/Sign Language classification/data/final_data"

In [5]:
df_train = pd.read_csv("/content/drive/MyDrive/competition/Zindi/Sign Language classification/data/Train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/competition/Zindi/Sign Language classification/data/Test.csv")

In [6]:
print("Number of training images", df_train.shape[0])
print("Number of testing images", df_test.shape[0])


Number of training images 6249
Number of testing images 2679


In [7]:
df_train.columns

Index(['img_IDS', 'Label'], dtype='object')

In [8]:
df_train["Label"].unique()

array(['Temple', 'Church', 'Enough/Satisfied', 'Me', 'Love', 'Mosque',
       'You', 'Friend', 'Seat'], dtype=object)

In [9]:
print(df_train["Label"].nunique())

9


# Process Dataset

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=40,
      width_shift_range=0.1,
      height_shift_range=0.1,
      brightness_range=[0.2,1.0],
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split=0.2 )

train_generator = train_datagen.flow_from_directory(IMAGE_TRAIN_DIR,
                                                    batch_size=30,
                                                    class_mode='categorical',
                                                    target_size=(224,224),
                                                    shuffle=True)


validation_generator = train_datagen.flow_from_directory(IMAGE_TRAIN_DIR,
                                                    batch_size=30,
                                                    class_mode='categorical',
                                                    target_size=(224,224),
                                                    shuffle=True,
                                                    subset="validation",
                                                    )




Found 6255 images belonging to 9 classes.
Found 1246 images belonging to 9 classes.


In [11]:
def preprocess_image_input(input_images):
  input_images = input_images.astype('float32')
  output_ims = tf.keras.applications.resnet50.preprocess_input(input_images)
  return output_ims


In [12]:
'''
Feature Extraction is performed by ResNet50 pretrained on imagenet weights. 
Input size is 224 x 224.
'''
def feature_extractor(inputs):

  feature_extractor = tf.keras.applications.resnet.ResNet50(input_shape=(224, 224, 3),
                                               include_top=False,
                                               weights='imagenet')(inputs)
  return feature_extractor


'''
Defines final dense layers and subsequent softmax layer for classification.
'''
def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(1024, activation="relu")(x)
    x = tf.keras.layers.Dense(9, activation="softmax", name="classification")(x)
    return x


def final_model(inputs):


    resnet_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(resnet_feature_extractor)

    return classification_output


def define_compile_model():
  inputs = tf.keras.layers.Input(shape=(224,224,3))
  
  classification_output = final_model(inputs) 
  model = tf.keras.Model(inputs=inputs, outputs = classification_output)
 
  # compile the model
  model.compile(optimizer=RMSprop(lr=0.0001),
              loss= tf.keras.losses.CategoricalCrossentropy() ,
              metrics=['acc'])

  return model


model = define_compile_model()

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 classification (Dense)      (None, 9)                 9225      
                                                             

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [13]:
EPOCHS = 15
checkpoint = ModelCheckpoint("/content/drive/MyDrive/competition/Zindi/Sign Language classification/data/model_weights2.h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint] 

history = model.fit(train_generator,validation_data =validation_generator, epochs=EPOCHS, callbacks=callbacks_list)

Epoch 1/15
209/209 [==============================] - ETA: 0s - loss: 0.8369 - acc: 0.7202
Epoch 00001: val_loss improved from inf to 2.54562, saving model to /content/drive/MyDrive/competition/Zindi/Sign Language classification/data/model_weights.h5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


209/209 [==============================] - 304s 1s/step - loss: 0.8369 - acc: 0.7202 - val_loss: 2.5456 - val_acc: 0.1108
Epoch 2/15
 28/209 [===>..........................] - ETA: 3:31 - loss: 0.4720 - acc: 0.8452

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-c1bad98ef839>", line 5, in <module>
    history = model.fit(train_generator,validation_data =validation_generator, epochs=EPOCHS, callbacks=callbacks_list)
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 64, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/engine/training.py", line 1216, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/eager/def_function.py", line 910, in __call__
    result = self._call(*args, **kwds)
  File "/usr/local/lib/python3.7/dist-packages/tens

KeyboardInterrupt: ignored

In [13]:
test_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_generator =  test_datagen.flow_from_directory( IMAGE_TEST_DIR,
                                                    shuffle=False,
                                                          batch_size  = 20,
                                                          class_mode  = None, 
                                                          target_size = (224, 224))

Found 2679 images belonging to 1 classes.


In [14]:
test_generator.reset()

In [15]:
pred = model.predict_generator(test_generator)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """Entry point for launching an IPython kernel.


In [16]:
print(pred.shape)
filenames=test_generator.filenames
print(len(filenames))

(2679, 9)
2679


In [17]:
res1 = list(map(lambda st: str.replace(st, "test/", ""), filenames))
res2 = list(map(lambda st: str.replace(st, ".jpg", ""), res1))


In [18]:
actual = train_generator.classes


In [19]:
type(pred)

numpy.ndarray

In [20]:
print(pred.shape)

(2679, 9)


In [21]:
import os
submission = pd.DataFrame()
submission["ID"] = res2
classes = ["Church","Enough/Satisfied","Friend","Love","Me","Mosque","Seat","Temple","You"]
for i, c in enumerate(classes):
  print(c)
  print(i)
  submission[c] = pred[:,i]
submission.head()

Church
0
Enough/Satisfied
1
Friend
2
Love
3
Me
4
Mosque
5
Seat
6
Temple
7
You
8


,ID,Church,Enough/Satisfied,Friend,Love,Me,Mosque,Seat,Temple,You
0,ImageID_00AVE728,1.184770e-13,1.736129e-17,4.802207e-17,1.421103e-21,2.224133e-20,2.472392e-22,1.529363e-21,1.000000e+00,2.524177e-22
1,ImageID_00CB7YJ2,5.568958e-17,1.000000e+00,1.236309e-17,1.859063e-20,9.652564e-22,1.281013e-21,5.642227e-21,8.569598e-21,1.425388e-20
2,ImageID_00HEGX6X,2.731526e-05,7.020454e-06,1.050166e-02,9.894543e-01,5.152945e-06,6.879222e-07,2.434829e-08,9.030932e-09,3.837778e-06
3,ImageID_016X4GBI,1.082680e-07,9.304893e-09,2.456886e-12,6.517807e-12,4.562535e-13,9.999999e-01,2.283030e-13,4.555290e-14,3.810057e-12
4,ImageID_01ITRYRU,1.410902e-10,1.240785e-06,1.136330e-06,5.601510e-07,1.025994e-10,5.585187e-07,9.999963e-01,4.245483e-09,9.841157e-08


In [26]:
submission.to_csv('/content/drive/MyDrive/competition/Zindi/Sign Language classification/data/sub4.csv',index=False)

In [23]:
submission.shape

(2679, 10)

In [24]:
submission[submission["ID"]=="ImageID_J3BM90F2"]

,ID,Church,Enough/Satisfied,Friend,Love,Me,Mosque,Seat,Temple,You
1420,ImageID_J3BM90F2,2.614250e-12,1.0,8.163008e-12,4.476683e-15,2.561795e-15,1.371805e-15,3.146961e-10,9.552415e-15,3.391745e-14
